<a href="https://colab.research.google.com/github/kauecapellato/artigo_ic/blob/main/PredicaoFutebol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bibliotecas

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Web Scraping

In [ ]:
def getTeams(URL_season):
    req = requests.get(URL_season)

    # Check if you have enterded the URL.
    if req.status_code == 200: 
        # Creates soup.
        soup = BeautifulSoup(req.text, 'html.parser')

        # Gets the table (table, in html) complete, by the "key" that is the name of the class. 
        teams_table = soup.find_all("table", class_="standard_tabelle")
        
        # Breaks the table string into the three rows (rows of the table). 
        teams_lines = str(teams_table[0]).split("<tr>")
        
        #  Dictionary whose key is the name of the time and the value is the url of it. 
        dict_names_url = {} 

        # Scrolls through the rows of the table and searches for the time name and its url, after which it saves in the dictionary above. 
        for column in teams_lines:
            if "td align=\"center\"" in column:
                team_name = column.split("href=\"")[1].split("\">")[1].split("alt=\"")[1].split("\"")[0]
                team_url = column.split("href=\"")[1].split("\">")[0]
                dict_names_url[team_name] = team_url

        return dict_names_url

In [ ]:
def getNotices(dict_names_url):
    URL_default = "https://www.worldfootball.net/"
    
    # Gets news for each team in the dictionary.
    for team_url in dict_names_url:
        # Creates the dictionary where the information will be.
        dict_notices = {'date': [], 'hour': [], 'title': [], 'subtitle': [], 'text': []}

        # Enter the first interaction of while.
        news_table = True 
        
        # Used to change the page number of the news. 
        cont_page = 1
        
        while news_table != []:
            try:
                # Gets the url with the informations according to cont_page
                req = requests.get(URL_default+dict_names_url[team_url].replace("/teams/", "news/"+str(cont_page)+"/"))
                
                # Check if you have enterded the URL.
                if req.status_code == 200:
                    # Creates soup.
                    soup1 = BeautifulSoup(req.text, 'html.parser')

                    # Gets table with the URLs in the news.
                    news_table = soup1.find_all("div", class_="wfb-news-general")
                    soup_news_table = BeautifulSoup(str(news_table[0]), 'html.parser')
                    news_list = soup_news_table.find_all("div", class_="module module-newmon module-newmon-default")
                    soup_news_list = BeautifulSoup(str(news_list[0]), 'html.parser')
                    news_list1 = soup_news_list.find_all("div", class_="module-newmon-image")

                    news_urls = []
                    # Stores news urls in news_urls. 
                    for news in news_list1:
                        soup_news_list1 = BeautifulSoup(str(news), 'html.parser')
                        news_url = soup_news_list1.find_all("a")
                        news_urls.append(str(news_url[0]).split("href=\"")[1].split("\"")[0])

                    # Each news_url has on average 20 different news. And each news one different URL.
                    for url in news_urls:
                        # Gets each news URL
                        url_news = url
                        req_news = requests.get(URL_default+url_news)

                        # Check if you have entered the url
                        if req_news.status_code == 200:
                            # Create soup.
                            soup2 = BeautifulSoup(req_news.text, 'html.parser')

                            # Gets the box where it is the date, subtitle and the news.
                            news_box = soup2.find_all("div", class_="box")                            
                            soup_box = BeautifulSoup(str(news_box[0]), 'html.parser')
                            news_date = soup_box.find_all("div", class_="wfb-news-date")
                            news_date = str(news_date[0]).split("\n\t\t")[1].split("h\n\t")[0]
                            news_date, news_hour, _ = news_date.split(" ")
                            
                            # Gets the title.
                            news_title = soup_box.find_all("h1", class_="wfb-news-title")
                            news_title = str(news_title[0]).split("\">")[1].split("</h1>")[0]
                            
                            # Find the box content, where it has the subtitle and the news. 
                            news_content = soup_box.find_all("div", class_="wfb-news-content")
                            news_paragraphs = BeautifulSoup(str(news_content[0]), 'html.parser')
                            news_paragraphs = news_paragraphs.find_all("p")

                            # Gets the subttitle.
                            news_subtitle = str(news_paragraphs[0]).split("<p>")[1].split("</p>")[0]
                            news_text = ""
                            
                            # Gets all paragraphs from the news. Disgard the 0 because is the subtitle.
                            for paragraph in range(1,len(news_paragraphs)):
                                news_text += str(news_paragraphs[paragraph]).split("<p>")[1].split("</p>")[0]
                                news_text += " "

                            # Appends date, hour, subtitle, title and text in the dictionary
                            dict_notices['date'].append(news_date.replace(".", "/"))
                            dict_notices['hour'].append(news_hour)
                            dict_notices['title'].append(news_title)
                            dict_notices['subtitle'].append(news_subtitle)
                            dict_notices['text'].append(news_text.strip())
                    # Next page
                    cont_page += 1
            except:
                # Next page
                cont_page += 1
                pass

        # Creates dataframe before takes the next team.
        df = pd.DataFrame(dict_notices, columns=['date', 'hour', 'title', 'subtitle', 'text'])      
        # path where to save .         
        PATH = f'C:\\Users\\Peterson\\Desktop\\IC\\{team_url}.csv'
        # Save as csv.
        df.to_csv(PATH)
        print(time_url + ": OK")